# TensorRT for DenseNet

This is a follow up notebook for optimizing_resnet, and repeats the experiment on denseNet. The DenseNet model is trained on Breast Cancer data that won the recent HumanVsAI hackathon.

DenseNet is State-of-The-Art network that require much fewer parameters to achieve the same performance as a traditional CNN, through feature reuse. As we can see below, each layer has access to feature maps from all previous layers, thus reducing generation of redundang feature maps.

![DenseNet Architecture](https://cdn-images-1.medium.com/max/600/1*GeK21UAbk4lEnNHhW_dgQA.png)

Image is taken from [link](https://towardsdatascience.com/understanding-and-visualizing-densenets-7f688092391a) Please read here for more information

In [1]:
# Import the libaries, we will start from Keras
import keras
from keras import backend as K
import time
import numpy as np
import os
from trt_utils import FrozenGraph,TrtModel, TfModel, tf_to_trt_graph, write_graph_to_file

currdir = os.getcwd()

Using TensorFlow backend.
W0621 00:01:16.570926 140012050593536 deprecation_wrapper.py:118] From /home/cheeyau/anaconda3/envs/tensorrt/lib/python3.6/site-packages/tensorflow/__init__.py:95: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0621 00:01:16.572038 140012050593536 deprecation_wrapper.py:118] From /home/cheeyau/anaconda3/envs/tensorrt/lib/python3.6/site-packages/tensorflow/__init__.py:95: The name tf.AttrValue is deprecated. Please use tf.compat.v1.AttrValue instead.

W0621 00:01:16.572964 140012050593536 deprecation_wrapper.py:118] From /home/cheeyau/anaconda3/envs/tensorrt/lib/python3.6/site-packages/tensorflow/__init__.py:95: The name tf.COMPILER_VERSION is deprecated. Please use tf.version.COMPILER_VERSION instead.

W0621 00:01:16.573601 140012050593536 deprecation_wrapper.py:118] From /home/cheeyau/anaconda3/envs/tensorrt/lib/python3.6/site-packages/tensorflow/__init__.py:95: The name tf.CXX11_ABI_FLAG is deprecated. Please use tf.sysco

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Keras Inference

In [3]:
from keras.models import load_model

In [4]:
model = load_model("densenet.h5")
batch_size = 128 # this needs to be specified in TensorRT inference model

# Generate test samples 
img_shape = (224, 224, 3)
x_test = np.random.random_sample((batch_size,img_shape[0], img_shape[1], img_shape[2]))


W0621 00:01:20.861027 140012050593536 deprecation_wrapper.py:118] From /home/cheeyau/anaconda3/envs/tensorrt/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0621 00:01:20.931249 140012050593536 deprecation_wrapper.py:118] From /home/cheeyau/anaconda3/envs/tensorrt/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0621 00:01:21.998417 140012050593536 deprecation_wrapper.py:118] From /home/cheeyau/anaconda3/envs/tensorrt/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0621 00:01:45.146929 140012050593536 deprecation_wrapper.py:118] From /home/cheeyau/anaconda3/envs/tensorrt/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Ple

In [5]:
y_keras = model.predict(x_test) # initialize the model first, don't take first predict into account
start = time.time()
y_keras = model.predict(x_test)
end = time.time()
Keras_time = end - start
print('Keras time : {0} s'.format(Keras_time) )

Keras time : 0.9828753471374512 s


## Tensorflow for Inference

Converting Keras to Tensorflow

In [6]:
frozen_graph = FrozenGraph(model, img_shape)

# Save the frozen file for reference
write_graph_to_file('dnet.pb',frozen_graph.frozen,currdir)

W0621 00:02:27.787399 140012050593536 deprecation.py:323] From /home/cheeyau/Git/Personal/TensorRT Next Extended/trt_utils.py:37: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0621 00:02:27.788107 140012050593536 deprecation.py:323] From /home/cheeyau/anaconda3/envs/tensorrt/lib/python3.6/site-packages/tensorflow_core/python/framework/graph_util_impl.py:270: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
W0621 00:02:31.002372 140012050593536 deprecation.py:323] From /home/cheeyau/Git/Personal/TensorRT Next Extended/trt_utils.py:38: remove_training_nodes (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
In

In [7]:
print('number of nodes for tf model: {0}'.format(len(frozen_graph.frozen.node)))

number of nodes for tf model: 1534


In [8]:
tf_model = TfModel(frozen_graph.frozen,frozen_graph.x_name,frozen_graph.y_name)

In [9]:
frozen_graph.x_name,frozen_graph.y_name

(['image_tensor_x'], ['model_1/dense_2/Softmax'])

In [10]:
#tf_model = TfEngine(frozen_graph)
y_tf = tf_model.infer(x_test)
t0 = time.time() 
y_tf = tf_model.infer(x_test)
t1 = time.time()
tf_time = t1-t0

In [11]:
print('Tensorflow time : {0} s'.format(tf_time) )

Tensorflow time : 0.9359209537506104 s


## TensorRT

Convert Tensorflow to tensorRT

In [12]:
imagenet_classes = 2
precision = "FP16"
x_name = ['image_tensor_x']
y_name =['model_1/dense_2/Softmax']

In [13]:
tftrt_graph_fp16 = tf_to_trt_graph(frozen_graph.frozen, y_name, batch_size, precision)

In [16]:
write_graph_to_file('dnet_fp16.pb',tftrt_graph_fp16,currdir)

In [17]:
trt_model = TrtModel(tftrt_graph_fp16, batch_size,x_name,y_name, imagenet_classes)

In [18]:
print('number of nodes for trt model: {0}'.format(len(tftrt_graph_fp16.node)))

number of nodes for trt model: 3


In [19]:
y_tftrt = trt_model.infer(x_test)
t0 = time.time() 
y_tftrt = trt_model.infer(x_test)
t1 = time.time()
trt_time = t1-t0

In [20]:
print('TensorRT time : {0} s'.format(trt_time) )

TensorRT time : 0.5131697654724121 s


In [21]:
print('Keras time : {0} s'.format(Keras_time) )
print('Tensorflow time : {0} s'.format(tf_time) )
print('TensorRT time : {0} s'.format(trt_time) )

Keras time : 0.9828753471374512 s
Tensorflow time : 0.9359209537506104 s
TensorRT time : 0.5131697654724121 s


Comparing results

In [22]:
def calculateErrorRate(result, ans):
    num_tests = ans.shape[0]
    error = 0
    for i in range(0, num_tests):
        a = np.argmax(ans[i])
        r = np.argmax(result[i])
        if (a != r) : error += 1
    
    error_rate = error/num_tests
    return error_rate

In [23]:
calculateErrorRate(y_tftrt,y_tf)

0.0